<a href="https://colab.research.google.com/github/JerryChenz/InvestmentManagement/blob/master/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Set inputs

We load the sample dataset from the my github repository, and display the dataset in pandas.

In [ ]:
import pandas as pd
summary_url = 'https://raw.githubusercontent.com/JerryChenz/InvestmentManagementOpen/main/financial_models/Opportunities/Screener/screener_summary.csv'
df = pd.read_csv(summary_url)
df = df.fillna(0)

# capitalization in reporting currency
capitalization_price = df['Price'] * df['Shares']
capitalization_report = capitalization_price * df['Fx_rate']
total_debt = df['CurrentDebtAndCapitalLeaseObligation'] + df['LongTermDebtAndCapitalLeaseObligation']
# more easily realizable non-operating assets
monetary_assets = df['CashAndCashEquivalents'] + df['OtherShortTermInvestments'] + df['InvestmentinFinancialAssets']
# less easily realizable non-operating assets
fixed_nonop_assets = (df['InvestmentProperties'] + df['LongTermEquityInvestment']) * 0.5
total_nonop_assets = monetary_assets + fixed_nonop_assets
enterprise_value = capitalization_report + total_debt + df['MinorityInterest'] - df['CashAndCashEquivalents'] - total_nonop_assets
# liquidity_coverage_ratio
lcr = monetary_assets / df['CurrentLiabilities']
current_ratio = df['CurrentAssets'] / df['CurrentLiabilities']
# dividend rate & buyback rate
dividend_rate = df['Dividend'] / df['Price']
buyback_rate = df['Buyback'] / df['Price']
# Net PPE/ Sales
ppe_multiple = df['NetPPE'] / df['TotalRevenue']

# display(df)

# Step 2: Screening Criteria

We can screen using different sets of conditions, then merge them later.

In [ ]:
# common fitlering conditions
common_1 = df['CashAndCashEquivalents'] > df['CurrentDebtAndCapitalLeaseObligation']
common_2 = lcr >= 0.8
common_3 = (total_debt / df['CurrentAssets']) < 1.5
common_4 = df['Avg_Gross_margin'] > 10
common_5 = capitalization_price > 1000000000

In [ ]:
# 1st set of conditions
condition_1 = capitalization_price > 6000000000
condition_2 = df['Avg_ebit_margin'] > 15

In [ ]:
# 2nd set of conditions
condition_5 = capitalization_price <= 6000000000
condition_6 = dividend_rate > 0.01
condition_7 = buyback_rate > 0.01

#Step 3. Output

Filter the dataset using the above conditions

In [ ]:
# filtered by common conditions
common_df = df
common_df['EV'] = enterprise_value
common_df['Dividend rate'] = dividend_rate
common_df['Buyback rate'] = buyback_rate
common_df = common_df.loc[common_1 & common_2 & common_3 & common_4]
display(common_df)

In [ ]:
# fitlered by 1st set of conditions
df_1 = common_df
df_1 = df_1.loc[condition_1 & condition_2]
print(df_1.to_string())

In [ ]:
# filtered by 2nd set of conditions
df_2 = common_df
df_2 = df_2.loc[condition_5 & (condition_6 | condition_7)]
print(df_2.to_string())

In [ ]:
# combine the results
result_set = pd.concat([df_1, df_2])
result_set = result_set.groupby("Ticker").first()
result_set = result_set.sort_values(by=['EV', 'Dividend rate'], ascending=[True, False]).reset_index()
print(result_set.to_string())

In [ ]:
# Listing Location
market = 'US' #@param ["HK","CN", "US"]

# HK only
if market == 'HK':
  exchange_condition = result_set['Exchange'] == 'HKG'
elif market == 'CN':
# A shares only
  exchange_condition = (result_set['Exchange'] == 'SHZ') | (result_set['Exchange'] == 'SHH')
else:
  exchange_condition = (result_set['Exchange'] == 'NMS') | (result_set['Exchange'] == 'NYQ')
display_set = result_set.loc[exchange_condition].sort_values(by=['EV', 'Dividend rate'], ascending=[True, False]).reset_index()
print(display_set.to_string())


    index Ticker                             Name Exchange     Price Price_currency        Shares Reporting_Currency   Fx_rate  Dividend    Buyback       Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate  Buyback rate
0       0   BABA    Alibaba Group Holding Limited      NYQ  114.8550            USD  2.647540e+09                CNY  0.147437  0.0